In [1]:
import dash 

import plotly.express as px
from dash import dcc
from dash import html
from dash.dependencies import Output, Input, State
import pandas as pd 
 


In [2]:
spoty_df = pd.read_csv('spotify_top100.csv')
spoty_df = spoty_df.dropna()

In [3]:
numeric_cols = spoty_df.select_dtypes(include=['float', 'int']).columns
spoty_df[numeric_cols] = spoty_df[numeric_cols].round(0).astype(int)
spoty_df[numeric_cols]

,year released,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop,top year
0,2009,140,81,61,-6,23,23,203,0,6,70,2010
1,2010,138,89,68,-4,36,83,192,1,8,68,2010
2,2010,95,48,84,-7,9,96,243,20,3,72,2010
3,2010,93,87,66,-4,4,38,180,11,12,80,2010
4,2010,104,85,69,-6,9,74,268,39,5,79,2010
...,...,...,...,...,...,...,...,...,...,...,...,...
995,2018,155,73,83,-4,12,45,313,1,22,86,2019
996,2019,80,50,55,-9,80,41,190,23,7,85,2019
997,2019,103,77,89,-5,9,46,177,1,7,68,2019
998,2019,138,58,53,-6,10,59,214,1,10,67,2019


In [4]:
spoty_df_list = list(spoty_df['top year'].unique())
marks_labels = {int(x) : str(x) for x in spoty_df_list}
marks_labels
spoty_df_list

[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

In [5]:
app = dash.Dash(__name__)

app.layout = html.Div(children=
                      [
                        html.H1(id='h1', children='Spotify top 100'),
                        dcc.Slider(id='slider',
                                     value=2010,
                                     step = 1,
                                     marks=marks_labels),
                                                
                        dcc.Graph(id='graph', 
                                  figure=px.bar(data_frame=spoty_df,
                                                x = list(spoty_df[['bpm', 'top year']].groupby('top year').agg('mean').index),
                                                y = list(spoty_df[['bpm', 'top year']].groupby('top year').agg('mean').values.squeeze()),
                                                labels=list(spoty_df[['bpm', 'top year']].groupby('top year').agg('mean').values.squeeze()))),

                        dcc.Dropdown(id='drop', 
                                     value= list(spoty_df[['nrgy', 'dB', 'live', 'val', 'dnce']].columns)[0],
                                     options= list(spoty_df[['nrgy', 'dB', 'live', 'val', 'dnce']].columns),
                                     placeholder='Select a category'

                                     )                    
                         
                      ])

@app.callback(
        Output(component_id='graph', component_property='figure'),
        Input(component_id='slider', component_property='value'),
        State(component_id='drop', component_property= 'value')
)

def update_function(slider_value, dropd_value):
    
    # Debugging: Zobrazení vstupních hodnot
    # print(f"Slider Value: {slider_value}")
    # print(f"Dropdown Value: {dropd_value}")
    
    numeric_columns = list(spoty_df[['nrgy', 'dB', 'live', 'val', 'dnce']].columns)

    
    filtered_df = spoty_df[spoty_df['top year'] == int(slider_value)]

    figure = px.bar(data_frame=filtered_df.head(15),
                    x = 'title', 
                    y = 'nrgy' if dropd_value == numeric_columns[0] else dropd_value,
                    height=800)
    
    layout_updates = dict(
        xaxis=dict(
        tickangle=45,
        automargin=True,
        categoryorder='total descending'))
    
    figure.update_layout(**layout_updates)
    
    return figure
    
if __name__ == '__main__':
    app.run(host='127.0.0.1', port= 3500, debug=True)


